## Descripción del dataset: Pima Indians Diabetes

El **Pima Indians Diabetes Dataset** es un conjunto de datos clásico en Machine Learning y bioestadística, recopilado por el *National Institute of Diabetes and Digestive and Kidney Diseases*.  
Su propósito es **predecir la aparición de diabetes tipo 2** en mujeres de origen **pima** (una población indígena del sur de Arizona, EE.UU.), a partir de diversas variables clínicas y demográficas.

### Características principales:
- **Número de registros:** 392 (en esta versión limpia, el original tenía 768).  
- **Número de atributos (features):** 8 variables predictoras + 1 variable objetivo.  
- **Población:** Mujeres de al menos 21 años de edad de la etnia Pima.  
- **Tarea principal:** Clasificación binaria → determinar si una paciente tiene diabetes (`Outcome = 1`) o no (`Outcome = 0`).

### Variables:
1. **Pregnancies** → Número de embarazos.  
2. **Glucose** → Concentración de glucosa en plasma después de 2 horas en una prueba de tolerancia a la glucosa.  
3. **BloodPressure** → Presión arterial diastólica (mm Hg).  
4. **SkinThickness** → Espesor del pliegue cutáneo del tríceps (mm).  
5. **Insulin** → Nivel sérico de insulina (mu U/ml).  
6. **BMI** → Índice de masa corporal (peso en kg / altura² en m²).  
7. **DiabetesPedigreeFunction** → Probabilidad de diabetes basada en antecedentes familiares.  
8. **Age** → Edad en años.  
9. **Outcome** → Variable objetivo:  
   - `0` = No tiene diabetes  
   - `1` = Tiene diabetes  

### Relevancia:
Este dataset es ampliamente utilizado en cursos de **Inteligencia Artificial y Machine Learning** para enseñar:
- Procesamiento y limpieza de datos biomédicos.  
- Métodos de clasificación supervisada (KNN, regresión logística, Random Forest, SVM, redes neuronales, etc.).  
- Importancia de la normalización y estandarización en algoritmos basados en distancias.  

---


## Paso 1: Cargar la base de datos  
Cargamos el CSV en un `DataFrame` de `pandas`. Si tu archivo no se llama exactamente `cleaned_dataset.csv`, ajusta la ruta.

In [4]:
import pandas as pd

df = pd.read_csv('dataset/cleaned_dataset.csv')
df.head()

,Pregnancies,Glucose,Blood Pressure,Skin Thickness,Insulin,BMI,Diabetes Pedigree Function,Age,Outcome
0,0,129,110,46,130,67.1,0.319,26,1
1,0,180,78,63,14,59.4,2.420,25,1
2,3,123,100,35,240,57.3,0.880,22,0
3,1,88,30,42,99,55.0,0.496,26,1
4,0,162,76,56,100,53.2,0.759,25,1


## Paso 2: Crear subconjuntos con 20 datos de **entrenamiento** y 20 de **testeo**
Seleccionaremos 40 muestras: 20 para entrenar y 20 para evaluar.

In [5]:
# Seleccionar 40 muestras aleatorias
subset = df.sample(n=40, random_state=42)

# Dividir en 20 entrenamiento y 20 testeo
train = subset.iloc[:20]
test = subset.iloc[20:]

train.head(), test.head()

(     Pregnancies  Glucose  Blood Pressure  Skin Thickness  Insulin   BMI  \
 78             2      146              76              35      194  38.2   
 274            7       83              78              26       71  29.3   
 246            0      120              74              18       63  30.5   
 55             0       91              68              32      210  39.9   
 387            1       92              62              25       41  19.5   
 
      Diabetes Pedigree Function  Age  Outcome  
 78                        0.329   29        0  
 274                       0.767   36        0  
 246                       0.285   26        0  
 55                        0.381   25        0  
 387                       0.482   25        0  ,
      Pregnancies  Glucose  Blood Pressure  Skin Thickness  Insulin   BMI  \
 15             0      118              84              47      230  45.8   
 388            1      100              74              12       46  19.5   
 345      

## Paso 3: Implementar la función de distancia euclidiana

**Instrucciones:**
- Escribe una función en Python que reciba dos vectores y calcule la distancia euclidiana entre ellos.
- Utiliza la siguiente fórmula matemática para la distancia euclidiana entre dos vectores $x$ y $y$ de $n$ dimensiones:

$$
d(x, y) = \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}
$$

- Prueba tu función con los siguientes dos ejemplos (cada vector corresponde a una fila del dataset):

| Embarazos | Glucosa | Presión Arterial | Grosor Piel | Insulina | IMC  | Función Hereditaria | Edad | Resultado |
|-----------|---------|------------------|-------------|----------|------|---------------------|------|-----------|
|     1     |   106   |        70        |      28     |   135    | 34.2 |        0.142        |  22  |     0     |
|     2     |   102   |        86        |      36     |   120    | 45.5 |        0.127        |  23  |     1     |

- Calcula la distancia euclidiana a mano y luego verifica que el resultado de tu función sea el mismo.
- La función debe imprimir el resultado del cálculo de la distancia euclidiana con los datos presentados.



In [6]:
import numpy as np

def euclidean_distance(x, y):
    return np.sqrt(np.sum((x - y) ** 2))

# Ejemplo de prueba:
vec1 = np.array([2, 146, 76, 35, 194, 38.2, 0.329, 29])
vec2 = np.array([0, 118, 84, 47, 230, 45.8, 0.551, 31])

dist = euclidean_distance(vec1, vec2)
dist

np.float64(48.51607242965984)

## Paso 4: Implementar un clasificador KNN básico

**Instrucciones:**
- Escribe una función que, dado un punto de prueba, calcule la distancia a todos los puntos de entrenamiento utilizando tu función de distancia euclidiana.
- Selecciona los **k = 3** vecinos más cercanos y predice la clase mayoritaria entre ellos.
- Aplica tu función a las 10 muestras de prueba obtenidas previamente, utilizando las 10 muestras de entrenamiento como referencia.
- El script debe imprimir una tabla comparando el valor real de `Resultado` de cada muestra de prueba con el valor predicho por tu algoritmo.
- Considere que las tablas se pueden codificar con un formato similar al que se muestra en el siguiente código:

In [7]:
# Implementar un clasificador KNN básico

def knn_predict(test_row, train_data, train_labels, k=3):
    distances = []
    for idx, row in train_data.iterrows():
        dist = euclidean_distance(test_row, row)
        distances.append((dist, train_labels.iloc[idx]))
    distances.sort(key=lambda x: x[0])
    neighbors = [label for _, label in distances[:k]]
    # Predicción: mayoría
    return max(set(neighbors), key=neighbors.count)

# Ejemplo de uso:
# test_row = test.iloc[0][:-1]  # Excluye Outcome
# train_data = train.iloc[:, :-1]
# train_labels = train['Outcome']
# pred = knn_predict(test_row, train_data, train_labels, k=3)
# pred

## Paso 5: Usar toda la data con separación 80% entrenamiento / 20% testeo  

### Pasos:
1. Cargar todo el dataset.  
2. Separar variables (X) y etiquetas (y).  
3. Aplicar `train_test_split` con 80% para entrenamiento y 20% para testeo.  
4. Mantener la proporción de clases usando estratificación.  
5. Guardar los conjuntos de datos para usarlos en KNN.  

In [8]:
# Separar el dataset completo en 80% entrenamiento y 20% testeo
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1]  # Todas las columnas menos Outcome
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((313, 8), (79, 8))

## Paso 6: Entrenar un KNN con los datos sin escalar (crudos) y calcular accuracy  

### Pasos:
1. Definir el valor de **k = 3** y el metodo **Euclidiano**.  
2. Entrenar el modelo KNN con los datos crudos (sin normalizar/estandarizar).  
3. Predecir las clases del conjunto de test.  
4. Calcular el **accuracy** comparando predicciones con etiquetas reales.  
5. Guardar el resultado para la tabla comparativa.  


In [9]:
# Entrenar y evaluar KNN con datos crudos (sin escalar)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn_raw = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn_raw.fit(X_train, y_train)
pred_raw = knn_raw.predict(X_test)

accuracy_raw = accuracy_score(y_test, pred_raw)
accuracy_raw

0.7088607594936709

## Paso 7: Normalizar (Min-Max scaling) y entrenar KNN, luego calcular accuracy  

### Pasos:
1. Aplicar **normalización Min-Max** a los datos de entrenamiento y test.  
2. Entrenar el modelo KNN con los datos normalizados.  
3. Predecir las clases del conjunto de test.  
4. Calcular el **accuracy** del modelo.  
5. Guardar el resultado para la tabla comparativa.  


In [10]:
# Normalizar datos con Min-Max scaling y entrenar KNN
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

knn_norm = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn_norm.fit(X_train_norm, y_train)
pred_norm = knn_norm.predict(X_test_norm)

accuracy_norm = accuracy_score(y_test, pred_norm)
accuracy_norm

0.7721518987341772

## Paso 9: Estandarizar (Z-score) y entrenar KNN, luego calcular accuracy  

### Pasos:
1. Aplicar **estandarización Z-score** a los datos de entrenamiento y test.  
2. Entrenar el modelo KNN con los datos estandarizados.  
3. Predecir las clases del conjunto de test.  
4. Calcular el **accuracy** del modelo.  
5. Guardar el resultado para la tabla comparativa.  


In [11]:
# Estandarizar datos con Z-score y entrenar KNN
from sklearn.preprocessing import StandardScaler

scaler_z = StandardScaler()
X_train_z = scaler_z.fit_transform(X_train)
X_test_z = scaler_z.transform(X_test)

knn_z = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn_z.fit(X_train_z, y_train)
pred_z = knn_z.predict(X_test_z)

accuracy_z = accuracy_score(y_test, pred_z)
accuracy_z

0.759493670886076

## Paso 10/11: Tabla comparativa de accuracies  

### Pasos:
1. Reunir los resultados de accuracy de cada experimento:  
   - KNN sin escalar (80/20).  
   - KNN normalizado (80/20).  
   - KNN estandarizado (80/20).  
2. Crear una tabla con los resultados.  
3. Comparar el desempeño de cada método.  



In [12]:
# Tabla comparativa de accuracies
import pandas as pd

results = pd.DataFrame({
    'Método': ['KNN crudo', 'KNN normalizado', 'KNN estandarizado'],
    'Accuracy': [accuracy_raw, accuracy_norm, accuracy_z]
})

results

,Método,Accuracy
0,KNN crudo,0.708861
1,KNN normalizado,0.772152
2,KNN estandarizado,0.759494


---
## Preguntas de reflexión y aplicación



1. ¿Por qué es importante normalizar o estandarizar los datos antes de usar KNN?  



Es importante normalizar o estandarizar los datos antes de usar KNN porque este algoritmo calcula distancias entre puntos. Si las variables tienen diferentes escalas, las de mayor rango dominarán el cálculo de distancia y el modelo será sesgado. Normalizar o estandarizar asegura que todas las variables contribuyan de manera equilibrada, mejorando la precisión y la interpretación del modelo.

2. ¿Qué diferencias observaste en el accuracy entre los datos crudos, normalizados y estandarizados?  


Al comparar los resultados, se observa que el accuracy mejora al normalizar y estandarizar los datos. El modelo KNN con datos crudos tiene menor precisión porque las variables con mayor rango afectan más el cálculo de distancia. Al aplicar Min-Max o Z-score, el modelo logra un mejor balance y predice con mayor exactitud, como se refleja en la tabla comparativa.

3. Si aumentamos el valor de **k** (número de vecinos), ¿cómo crees que cambiaría el rendimiento del modelo?  


Si aumentamos el valor de k, el modelo se vuelve menos sensible a valores atípicos y el resultado depende más de la mayoría de los vecinos. Esto puede mejorar la estabilidad y reducir el sobreajuste, pero si k es demasiado grande, el modelo puede perder capacidad de distinguir clases minoritarias y disminuir su precisión.

4. ¿Qué ventaja tiene implementar KNN manualmente antes de usar scikit-learn?  


Implementar KNN manualmente permite comprender a fondo cómo funciona el algoritmo, cómo se calculan las distancias y cómo se seleccionan los vecinos. Esto ayuda a detectar posibles errores, ajustar el método según el problema y aprender los fundamentos antes de usar herramientas automáticas como scikit-learn.

5. ¿Qué limitaciones presenta KNN cuando se aplica a conjuntos de datos grandes o con muchas dimensiones?  

KNN presenta limitaciones en conjuntos de datos grandes porque requiere calcular distancias entre todos los puntos, lo que es costoso en tiempo y memoria. Además, en datos con muchas dimensiones, la distancia pierde significado (problema de la "maldición de la dimensionalidad"), lo que puede reducir la precisión y dificultar la clasificación.

---

## Rúbrica de evaluación: Práctica KNN

| Criterio | Descripción | Puntaje Máximo |
|----------|-------------|----------------|
| **1. Carga y exploración del dataset** | Carga correcta del archivo CSV, explicación de las variables y verificación de datos. | 15 pts |
| **2. Implementación manual de KNN** | Código propio para calcular distancias euclidianas, selección de vecinos y votación mayoritaria. | 20 pts |
| **3. Predicción individual (ejemplo aleatorio)** | Explicación clara del proceso paso a paso para un ejemplo de test. | 10 pts |
| **4. Uso de scikit-learn (KNN)** | Entrenamiento y evaluación con `train_test_split`, comparación con el método manual. | 15 pts |
| **5. Normalización y estandarización** | Aplicación correcta de Min-Max y Z-score, con cálculo de accuracy en cada caso. | 20 pts |
| **6. Tabla comparativa de accuracies** | Presentación clara de los resultados y comparación entre métodos. | 10 pts |
| **7. Reflexión y preguntas finales** | Respuestas a las preguntas de análisis planteadas (profundidad y claridad). | 10 pts |

**Total: 100 pts**
